<a href="https://colab.research.google.com/github/Yagwar/Sociadisticas/blob/master/Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
import networkx as nx
import multiprocessing as mp
import csv


import time
import seaborn as sns
sns.set(color_codes=True)

import matplotlib.gridspec as gridspec
from scipy.cluster import hierarchy

import matplotlib as mpl
mpl.style.use('seaborn-talk')
%matplotlib inline

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
drive_path = 'gdrive/My Drive/Freelancer/alc_bog_19/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
def tweet_extrac(tweet):
  tweet_id = tweet["id_str"] # Identificador del tweet
  date_tweet=tweet["created_at"]
  user_id = tweet["user"]["id_str"] # Identificador del usuario
  user_name = tweet["user"]["screen_name"] # Nombre del usuario
  user_location = tweet["user"]["location"]
  user_verified = tweet["user"]["verified"]
  user_followers_count = tweet["user"]["followers_count"]
  user_friends_count = tweet["user"]["friends_count"]
  user_favourites_count = tweet["user"]["favourites_count"]
  user_statuses_count = tweet["user"]["statuses_count"]
  user_created_at = tweet["user"]["created_at"]
  is_retweet=False

  tw_text=tweet["text"]
  if tweet["truncated"]:#=="True": # Truncated indica si el tweet es corto o largo; si truncado, busca extendido
      tw_text=tweet["extended_tweet"]["full_text"]
  else:
      if tweet["text"][0:2]=="RT":
          is_retweet=True
          try:
            if tweet["retweeted_status"]["truncated"]:
              tw_text=tweet["retweeted_status"]["extended_tweet"]["full_text"]
            else: 
                tw_text=tweet["retweeted_status"]["text"]
          except:
            tw_text=tweet["text"]
  is_quote_status=tweet["is_quote_status"] #True/False
  user_id_replyed=tweet['in_reply_to_user_id']#
  user_name_replyed=tweet["in_reply_to_screen_name"]
  tweet_id_replyed=tweet['in_reply_to_status_id']
  quote_count=tweet['quote_count']
  reply_count=tweet['reply_count']
  retweet_count=tweet['retweet_count']
  retweeted=tweet["retweeted"]
  tweet_url="https://twitter.com/"+user_name+"/status/"+tweet_id
  is_reply=False
  if not(tweet['in_reply_to_user_id']==None):
    is_reply=True

  return {"tweet_id":tweet_id,
          "date_tweet":date_tweet,
          "user_id":user_id,
          "user_name":user_name,
          "user_location":user_location,
          "user_verified":user_verified,
          "user_followers_count":user_followers_count,
          "user_friends_count":user_friends_count,
          "user_favourites_count":user_favourites_count,
          "user_statuses_count":user_statuses_count,
          "user_created_at":user_created_at,
          "is_retweet":is_retweet,
          "tw_text":tw_text,
          "is_quote_status":is_quote_status,
          "is_reply":is_reply,
          "user_id_replyed":user_id_replyed,
          "user_name_replyed":user_name_replyed,
          "tweet_id_replyed":tweet_id_replyed,
          "quote_count":quote_count,
          "reply_count":reply_count,
          "retweet_count":retweet_count,
          "retweeted":retweeted,
          "tweet_url":tweet_url
         }

# def line2csv (line, output_file, write_param="a"):#a append # r write new file
#   dict_extracted=""
#   try:
#     tweet = json.loads(line)
#     dict_extracted=tweet_extrac(tweet)
#     df_extracted=pd.DataFrame([dict_extracted])
#     with open(output_file, write_param) as f:
#       writer = csv.writer(f)
#       writer.writerow(list(dict_extracted.values()))
#   return dict_extracted

# def tw2csv (tweet, output_file, write_param="a"):#a append # r write new file
#   dict_extracted=tweet_extrac(tweet)
#   df_extracted=pd.DataFrame([dict_extracted])
#   with open(output_file, write_param) as f:
#     writer = csv.writer(f)
#     writer.writerow(list(dict_extracted.values()))
#   return dict_extracted

In [0]:
tweets_datas_path = [
    drive_path+"candidatos_alcaldia_7.json",
    drive_path+"candidatos_alcaldia_8.json",
    drive_path+"candidatos_alcaldia_9.json",
    drive_path+"candidatos_alcaldia_10.json",
    drive_path+"candidatos_alcaldia_11.json"
]

# Plan de trabajo
- Extraer las variables de interés
- Identificar si dentro de las menciones están los candidatos a la alcaldía.
- Integrar en un data frame la extracción.

In [0]:
cuentas=["ClaudiaLopez","carlosfgalan","angelamrobledo","MiguelUribeT","angelagarzonc",
         "luchogarzon","hollmanmorris","MAndreaNieto","CelioNievesH","jerojasrodrigue",
         "LuisErnestoGL"]
ids_cands=["137908875","64791701","165748292","163341528","325856294",
           "60161414","87266285","999594384","240409671","233001872",
           "2499219806"]

# Complete Tweets Extractor


In [0]:
# tweet_complete=[]
# extracts_twts=[]
# mentioned_ids=[]

In [0]:
completed. Total time: time_1_read=time.time()
# i=0

for json_file in tweets_datas_path:
  with open(json_file, "r") as tweets_file:
    for line in tweets_file:
#       i+=1
#       if i>200:#00:
#         break
      try:
        tw_entities=[]
        tweet = json.loads(line)
#         tweet_complete.extend([tweet])
        tw_entities.extend(tweet["entities"]["user_mentions"])
        if tweet["text"][0:2]=="RT":
          try:
            if tweet["retweeted_status"]["truncated"]:
              tw_entities.extend(tweet["retweeted_status"]["extended_tweet"]["entities"]["user_mentions"])
            else:
              tw_entities.extend(tweet["retweeted_status"]["entities"]["user_mentions"])
          except:
            tw_entities.extend(tweet["entities"]["user_mentions"])
        extracted=tweet_extrac(tweet)
#         extracts_twts.append(extracted)
        mentions_info=[tweet["id_str"],
                       tweet["user"]["screen_name"],
                       list(set([ent["id_str"] for ent in tw_entities])),
                       list(set([ent["screen_name"] for ent in tw_entities]))]
        out_mentions_cands=[tweet["id_str"]]
        out_mentions_cands.extend([cand_id in mentions_info[2] for cand_id in ids_cands])
        with open(drive_path+'cands_mentions.csv', 'a', newline='') as cnd_mnts:
          writer = csv.writer(cnd_mnts)
          writer.writerow(out_mentions_cands)#tweet["id_str"],
        with open(drive_path+'mentions.csv', 'a', newline='') as ments_gral:
          writer = csv.writer(ments_gral)
          writer.writerow(mentions_info)
        with open(drive_path+'extracted_dict.csv', 'a', newline='') as ext_dict:
          writer = csv.writer(ext_dict)
          writer.writerow(list(extracted.values()))
      except:
        continue
  print(json_file,"completed. Total time:", str(time.time()-time_1_read),"seconds; ",)
# print(str(time.time()-time_1_read),"seconds; ",str(len(tweet_complete)),"registros")

gdrive/My Drive/Freelancer/alc_bog_19/candidatos_alcaldia_7.json completed. Total time: 3843.2697072029114 seconds; 
gdrive/My Drive/Freelancer/alc_bog_19/candidatos_alcaldia_8.json completed. Total time: 7516.527146577835 seconds; 
gdrive/My Drive/Freelancer/alc_bog_19/candidatos_alcaldia_9.json completed. Total time: 11047.429228305817 seconds; 
gdrive/My Drive/Freelancer/alc_bog_19/candidatos_alcaldia_10.json completed. Total time: 14595.582593202591 seconds; 
gdrive/My Drive/Freelancer/alc_bog_19/candidatos_alcaldia_11.json completed. Total time: 18407.491209745407 seconds; 


    gdrive/My Drive/Freelancer/alc_bog_19/candidatos_alcaldia_7.json completed. Total time: 3843.2697072029114 seconds; 
    gdrive/My Drive/Freelancer/alc_bog_19/candidatos_alcaldia_8.json completed. Total time: 7516.527146577835 seconds; 
    gdrive/My Drive/Freelancer/alc_bog_19/candidatos_alcaldia_9.json completed. Total time: 11047.429228305817 seconds; 
    gdrive/My Drive/Freelancer/alc_bog_19/candidatos_alcaldia_10.json completed. Total time: 14595.582593202591 seconds; 
    gdrive/My Drive/Freelancer/alc_bog_19/candidatos_alcaldia_11.json completed. Total time: 18407.491209745407 seconds; 

In [0]:
# time_1_extract=time.time()
# i_process=0
# for tweet in tweet_complete:
#   extracts_twts.append(tweet_extrac(tweet))
#   i_process+=1
# print(str(time.time()-time_1_extract),"seconds; ",str(len(extracts_twts)),"registros")

In [0]:
# dats_extracted=pd.DataFrame(extracts_twts)

In [0]:
# print(dats_extracted.shape)
# dats_extracted.sample(20)

In [0]:
# dats_extracted.to_csv(drive_path+"candidatos_alcaldia_2.csv")

# Second version


In [0]:
# tweet=tweet_complete[199]
# tw_entities=[]

In [0]:
# xtrct=tweet_extrac(tweet)
# list(xtrct.values())
# # list(xtrct.keys())
# # list(tweet_extrac(.values())

['1171925765283307521',
 'Wed Sep 11 23:17:30 +0000 2019',
 '296637311',
 'kikearcinieg',
 None,
 False,
 1180,
 4957,
 30013,
 42647,
 'Wed May 11 04:23:37 +0000 2011',
 True,
 '¿saben por qué @EnriquePenalosa le solicita a la Contraloría General asumir la auditoría a la Empresa Metro? Porque así le quita la competencia de investigar a la Contraloría de Bogotá. Es una forma de detener la investigación en curso. @HOLLMANMORRIS https://t.co/8qSGxdzMR6',
 False,
 False,
 None,
 None,
 None,
 0,
 0,
 0,
 False,
 'https://twitter.com/kikearcinieg/status/1171925765283307521']

In [0]:
# tw_entities.extend(tweet["entities"]["user_mentions"])
# if tweet["text"][0:2]=="RT":
#     try:
#       if tweet["retweeted_status"]["truncated"]:
#         tw_entities.extend(tweet["retweeted_status"]["extended_tweet"]["entities"]["user_mentions"])
#       else: 
#           tw_entities.extend(tweet["retweeted_status"]["entities"]["user_mentions"])
#     except:
#       tw_entities.extend(tweet["entities"]["user_mentions"])


In [0]:
# [tweet["id_str"],
#  tweet["user"]["screen_name"],
#  list(set([ent["id_str"] for ent in tw_entities])),
#  list(set([ent["screen_name"] for ent in tw_entities]))
# ]

['1171925765283307521',
 'kikearcinieg',
 ['108717093', '66172488', '87266285'],
 ['EnriquePenalosa', 'MaPaulaFonseca', 'HOLLMANMORRIS']]

In [0]:
# list(set([ent["screen_name"] for ent in tw_entities]))

[]

In [0]:
# tw_entities.extend(tweet["entities"]["user_mentions"])


[{'id': 108717093,
  'id_str': '108717093',
  'indices': [15, 31],
  'name': 'Enrique Peñalosa',
  'screen_name': 'EnriquePenalosa'},
 {'id': 87266285,
  'id_str': '87266285',
  'indices': [237, 251],
  'name': 'Hollman Morris 🚇🌳👩🏽\u200d🎓',
  'screen_name': 'HOLLMANMORRIS'}]

In [0]:
# tweet["retweeted_status"]["entities"]["user_mentions"]

[{'id': 108717093,
  'id_str': '108717093',
  'indices': [15, 31],
  'name': 'Enrique Peñalosa',
  'screen_name': 'EnriquePenalosa'}]

In [0]:
# tot_ents=[]
# for tweet in tweet_complete:
#   tw_entities=[]
#   tw_entities.extend(tweet["entities"]["user_mentions"])
#   if tweet["text"][0:2]=="RT":
#     try:
#       if tweet["retweeted_status"]["truncated"]:
#         tw_entities.extend(tweet["retweeted_status"]["extended_tweet"]["entities"]["user_mentions"])
#       else: 
#         tw_entities.extend(tweet["retweeted_status"]["entities"]["user_mentions"])
#     except:
#       tw_entities.extend(tweet["entities"]["user_mentions"])
#   tot_ents.append([tweet["id_str"],
#                    tweet["user"]["screen_name"],
#                    list(set([ent["id_str"] for ent in tw_entities])),
#                    list(set([ent["screen_name"] for ent in tw_entities]))
#                   ])

In [0]:
# tot_ents[150]#[2]

['1171924904591142913',
 'NanyPardo',
 ['137908875', '163341528', '64791701', '534740565', '87266285'],
 ['MiguelUribeT', 'CarlosFGalan', 'ClaudiaLopez', 'carjotaz', 'HOLLMANMORRIS']]

In [0]:
# cuentas
# ids_cands

['ClaudiaLopez',
 'carlosfgalan',
 'angelamrobledo',
 'MiguelUribeT',
 'angelagarzonc',
 'luchogarzon',
 'hollmanmorris',
 'MAndreaNieto',
 'CelioNievesH',
 'jerojasrodrigue',
 'LuisErnestoGL']

In [0]:
# # np.array([[cand_id in ments_list[2] for cand_id in ids_cands]for ments_list in tot_ents]).shape
# out_test=[tweet["id_str"]]
# out_test.extend([cand_id in tot_ents[150][2] for cand_id in ids_cands])#.extend(tweet["id_str"])

In [0]:
# out_test

['1171925765283307521',
 True,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 False]

['1171925765283307521']

In [0]:
# tweet["id_str"]

'1171925765283307521'